In [1]:
import pandas as pd
import numpy as np
import sklearn
import os
from scipy.sparse import csc_matrix
# для встроенных картинок
%pylab inline
# %matplotlib inline
# чуть покрасивше картинки:
pd.set_option('display.mpl_style', 'default')

os.chdir("C:/htdocs/Walmart")
datatrain = pd.read_csv('input/train.csv')
datatest = pd.read_csv('input/test.csv')

Populating the interactive namespace from numpy and matplotlib


In [2]:
#df['Col3'] = df['Col2'].map(lambda x: 42 if x > 1 else 55)
#datatrain['isTrain']=True
#datatest['isTrain']=False
data=pd.concat([datatrain, datatest])
data['FinelineNumber'].fillna(-1, inplace= True)
data['DepartmentDescription'].fillna('n', inplace= True)

In [3]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [4]:
sp = csc_matrix( (1,1), dtype=np.float64 )

In [8]:

unique(data['Weekday'])

array(['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
       'Wednesday'], dtype=object)

In [3]:
date

,DepartmentDescription,FinelineNumber,ScanCount,TripType,Upc,VisitNumber,Weekday
0,FINANCIAL SERVICES,1000,-1,999,68113152929,5,Friday
1,SHOES,8931,1,30,60538815980,7,Friday
2,PERSONAL CARE,4504,1,30,7410811099,7,Friday
3,PAINT AND ACCESSORIES,3565,2,26,2238403510,8,Friday
4,PAINT AND ACCESSORIES,1017,2,26,2006613744,8,Friday
5,PAINT AND ACCESSORIES,1017,2,26,2006618783,8,Friday
6,PAINT AND ACCESSORIES,1017,1,26,2006613743,8,Friday
7,PAINT AND ACCESSORIES,2802,1,26,7004802737,8,Friday
8,PAINT AND ACCESSORIES,4501,1,26,2238495318,8,Friday
9,PAINT AND ACCESSORIES,3565,-1,26,2238400200,8,Friday


In [7]:
for i in (unique(data['Weekday'])):
    ScanCount_on_i=data
    ScanCount_on_i.loc[ScanCount_on_i['Weekday']!=i, 'ScanCount']=0
    DepartmentDescription = le.fit_transform(ScanCount_on_i['DepartmentDescription'])
    FinelineNumber = le.fit_transform(ScanCount_on_i['FinelineNumber'])
    SparceTmp = csc_matrix((ScanCount_on_i.ScanCount, (ScanCount_on_i.VisitNumber, DepartmentDescription))).astype(np.float64)
    SparceTmp2 = csc_matrix((ScanCount_on_i.ScanCount, (ScanCount_on_i.VisitNumber, FinelineNumber))).astype(np.float64)
    print(SparceTmp.shape)
    if sp.shape[0]  < 2:
        sp = SparceTmp.toarray()
    else:
        print(i)
        sp=hstack([sp, SparceTmp.toarray(), SparceTmp2.toarray()])
    

(191349, 69)
(191349, 69)
Monday


MemoryError: 

In [68]:
sp.shape

(191349, 483)

In [7]:
data[data['VisitNumber']==8].groupby('DepartmentDescription')['ScanCount'].sum()

DepartmentDescription
DAIRY                        2
DSD GROCERY                  1
HOUSEHOLD CHEMICALS/SUPP     1
MEAT - FRESH & FROZEN        1
PAINT AND ACCESSORIES       18
PETS AND SUPPLIES            4
n                            1
Name: ScanCount, dtype: int64

In [8]:
#gr=data.groupby(['DepartmentDescription', 'VisitNumber'])['ScanCount'].sum()
#'size' : size
gr=data.groupby(['DepartmentDescription', 'VisitNumber'])['ScanCount'].agg({'ScanCount_mean' : np.mean,  
                                                                            'ScanCount_max' : np.max, 'ScanCount_sum' : np.sum})
gr=gr.reset_index()

In [9]:
gr[gr['VisitNumber']==8]
#print(data.shape,gr.shape)

,DepartmentDescription,VisitNumber,ScanCount_sum,ScanCount_max,ScanCount_mean
101766,DAIRY,8,2,2,2.000
137799,DSD GROCERY,8,1,1,1.000
313050,HOUSEHOLD CHEMICALS/SUPP,8,1,1,1.000
433427,MEAT - FRESH & FROZEN,8,1,1,1.000
468845,PAINT AND ACCESSORIES,8,18,2,1.125
506702,PETS AND SUPPLIES,8,4,3,2.000
643448,n,8,1,1,1.000


In [10]:
gr2=gr.pivot(index='VisitNumber', columns='DepartmentDescription', values='ScanCount_mean')
gr2.fillna(0, inplace= True)

In [11]:
gr2

DepartmentDescription,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,BOYS WEAR,BRAS & SHAPEWEAR,...,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS,n
VisitNumber,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1]:
from scipy.sparse import csr_matrix
S3=csr_matrix(gr2.values).astype(np.float64)
print(gr2.shape, S3.shape)

NameError: name 'gr2' is not defined

In [75]:
# вектор ответов и индексы обучения
y = data[~data['TripType'].isnull()].groupby('VisitNumber')['TripType'].first()
indextrain = y.index.values
y = y.values
 
encodery = LabelEncoder()
y = encodery.fit_transform(y)

In [70]:
test_x = data[data['TripType'].isnull()].groupby('VisitNumber')['TripType'].first()
indextest = test_x.index.values

In [19]:
#indextrain
indextest

array([     1,      2,      3, ..., 191340, 191341, 191348], dtype=int64)

In [71]:
 
# контроль по фолдам - логистическая регрессия
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack

In [20]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [9]:
#from sklearn.decomposition import TruncatedSVD
#svd = TruncatedSVD(n_components=1000, random_state=42)
#S2=svd.fit_transform(S2) 

In [76]:
#sp.shape
y.shape

(95674,)

In [78]:
#print (cross_val_score(LogisticRegression(), hstack([S1[indextrain,:], S2[indextrain,:], S3[indextrain,:]]), 
#                       y, cv=StratifiedKFold(y, n_folds=2), scoring='log_loss'))

print (cross_val_score(LogisticRegression(), sp[indextrain,:], 
                       y, cv=StratifiedKFold(y, n_folds=2), scoring='log_loss'))

#clf = Pipeline([('svc', svm.SVC(C=1.0))])
#print (cross_val_score(LinearSVC(C=1.0), hstack([S1[indextrain,:], S2[indextrain,:]]), 
#                       y, cv=StratifiedKFold(y, n_folds=2), scoring='log_loss'))



[-3.07552739 -3.07491186]


In [23]:
sub = pd.read_csv('input/sample_submission.csv')

In [24]:
lr=LogisticRegression(
            #C=1.0, penalty='l2'
            #solver="lbfgs", multi_class='multinomial'
            )

In [25]:
lr.fit(hstack([S1[indextrain,:], S2[indextrain,:], S3[indextrain,:]]), y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [26]:
res=lr.predict_proba(hstack([S1[indextest,:], S2[indextest,:], S[indextest,:]]))

In [207]:
############## SVM #######################
from sklearn.svm import LinearSVC
svc = LinearSVC(C=1.0)
svc.fit(hstack([S1[indextrain,:], S2[indextrain,:]]), y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [209]:
res=svc.decision_function(hstack([S1[indextest,:], S2[indextest,:]]))

In [210]:
res.shape

(95674, 38)

In [27]:
my_sub=np.column_stack([indextest, res])
my_sub_df=pd.DataFrame(my_sub)
my_sub_df.columns=sub.columns
#my_sub_df.shape
my_sub_df['VisitNumber']=my_sub_df['VisitNumber'].apply("{0:.0f}".format)

In [28]:
my_sub_df.to_csv('output/Upc.csv', index=False) 

import numpy as np
from scipy.sparse import csc_matrix
csc_matrix((3, 4), dtype=np.int8).toarray()

In [23]:
row = np.array([0,1,2,0])
col = np.array([1,1,1,1])
data = np.array([1, 2, 3,2])
csc_matrix((data, (row, col))).toarray()

array([[0, 3],
       [0, 2],
       [0, 3]], dtype=int32)